# Installing and Importing and Initializing Libraries.

In [ ]:
!pip install pandarallel

In [ ]:
import numpy as np # linear algebra
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)
from pandarallel import pandarallel
import plotly.express as px
pd.set_option('display.max_row', 500)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from tqdm import tqdm
tqdm.pandas()
pandarallel.initialize(progress_bar=True)

# Reading the Datasets.

In [ ]:
df_series=pd.read_parquet(path="/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet", engine='auto')

In [ ]:
df_events=pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
events=df_events[['series_id', 'step','event']]

# Merging the Datasets to train the model

In [ ]:
series_df=pd.merge(df_series,events,on=["step","series_id"],how='left')
series_df['sleep']=np.nan
series_df.loc[series_df["event"]=="onset", "sleep"] = 1
series_df.loc[series_df["event"]=="wakeup", "sleep"] = 0
series_df['sleep'].fillna(method='ffill', inplace=True)
series_df['sleep'].fillna(value=0, inplace=True)
df_series=[]

# Windowing the training data
### 30min window before and after the event. 

In [ ]:
def window(df):
    ind=df.index[df['event'].isna()==False]
    c=0
    for i in tqdm(ind):
        a=i-360
        b=i+360
        df['window'].loc[a:i]=int(c)
        c=c+1
        df['window'].loc[i:b]=int(c)
        c=c+1
    print('Windows Created: ', c)
    df['window'].dropna(inplace=True)
    return df
series_df=window(series_df)


# Clustering the Enmo and Anglez

In [ ]:
def clustering(df):
    from sklearn.cluster import KMeans
    from sklearn.preprocessing import StandardScaler    
    X=df[['anglez','enmo']]
#Scalling the data
    scaler=StandardScaler()
    X_scaled=scaler.fit_transform(X)
#perform clustering
    model=KMeans(n_clusters=4, verbose=1, )
    model.fit(X_scaled)
    return model.labels_
series_df['cluster']=clustering(series_df)

# Adding Rolling Standard Deviations

- As per observation enmo and anglez varies frequently and more than normal for few timesteps before and after the event.
- In order to consider the variations into the modelling, using the method.

In [ ]:
def rollingstd(df):
    df['sd_anglez_1']=pd.to_numeric(df['anglez'].rolling(window=12).std())
    df['sd_enmo_1']=pd.to_numeric(df['enmo'].rolling(window=12).std())
    df['sd_anglez_1'].fillna('0.0',inplace=True)
    df['sd_enmo_1'].fillna('0.0',inplace=True)
    df['sleep'] = df['sleep'].replace({0:False, 0.2:True})
    return(df)
series_df=rollingstd(series_df)